## Gradient Methods

### Method of Steepest Descent

This method is fairly simple. We simply iterate in the opposite direction of the gradient according to some parameter $\lambda_k$. The method is described here:
$$x_{n+1} = x_{n} - \lambda_k \times \nabla f(\bold{x})$$
The question is, how do we define $\lambda_{k}$? One possible way is the Armijio rule. This rule sets $\lambda_k$ equal to some constant $\alpha^n$ such that $|f(x_{n+1} - f(x_{n})| \leq threshold$. The code below implements the algorithm

In [142]:
using LinearAlgebra
using ForwardDiff
using Random

function armijo(f::Function, guess::Vector{Float64}; beta::Float64 = 0.9, 
    threshold::Float64 = 1e-8, tol::Float64 = 0.001, max_iter::Int64 = 50)

    gradient = x -> ForwardDiff.gradient(f, x)
    
    #Intialize Guess_Array
    Guess_Array = Array{Any}(undef, max_iter+1)
    Guess_Array[1,1] = guess

    for i in 2:max_iter+1
        j::Int64 = 0
        val = f(Guess_Array[i-1])
        grad = gradient(Guess_Array[i-1])
        beta_n::Float64 = 1.0
        guess = Guess_Array[i-1] - grad
        while true
            if f(Guess_Array[i-1])-f(guess) >= threshold
                break
            end
            j += 1
            guess = Guess_Array[i-1] - beta ^ j .* grad
            if j >= 50
                println("""Iteration stalled \n
                           Current Derivative Error: $(norm(gradient(guess),2))\n
                           Current Guess: $(guess)\n
                        """)
                return Guess_Array[1:i-1]
            end
        end
        Guess_Array[i] = guess
        if norm(gradient(guess),2) <= tol
            return Guess_Array[1:i]
        end
    end
    println("""Tolerance not achieved\n 
    Current Derivative Error: $(norm(Guess_Array[end],2))\n
    Current Guess: $(Guess_Array[end])\n
    """)
    return Guess_Array
end

function ex(x::Vector)
    k = x'*x
    return exp(k)
end

ex (generic function with 1 method)

In [73]:
nums = 2 .* rand(Float64, 10) .- 1
nums = nums/norm(nums,10)
println([norm(x,2) for x in armijo(ex,nums; max_iter = 1000)])

[1.8108464223615186, 1.4912905211036418, 1.2237452293613427, 1.029131280571205, 0.8335327368108912, 0.7637726768186607, 0.6909971754049301, 0.6244835319759886, 0.5857997867881243, 0.49758402227999116, 0.4317051138901623, 0.4109335231417902, 0.37724448808648803, 0.3273574664051822, 0.26294898033546915, 0.24424262571485447, 0.22241843963393362, 0.19823827574230615, 0.17289261249984375, 0.147757066699596, 0.12407603997195384, 0.10272567463866702, 0.08414210598869953, 0.0683897805624674, 0.055288939187099206, 0.044535836463664435, 0.035787828505844346, 0.02871282033501706, 0.023012882711552475, 0.01843224940058789, 0.014757073605259999, 0.011811444112811003, 0.009452121567887606, 0.007563217381060926, 0.006051352666711962, 0.0048414810093168085, 0.0038733890810771217, 0.003098815869064706, 0.002479106257883114, 0.0019833124321039603, 0.0015866639882584847, 0.0012693383805911598, 0.001015474385805815, 0.0008123813935070427, 0.0006499060798617164, 0.0005199253580002299, 0.0004159405393856445

The example code above shows a steady decrease in the error as expected. 

### Line Search Methods And Descent Direction

A vector $d$ is said to be in the descent direction if $\nabla f(x)^T d < 0$. For the steepest descent method, we have that $d = [-1, -1, -1, \cdots -1]^T$. For newtons method, we have $d = -1 \times \nabla^2f(x_{n})^{-1} \nabla f(x)$. Howerver, if the hessian is not semipositive definite, the newton step may fail to be a descent direction. In general, a method that satisfies the armijo rule with the descent direction satisfying $d = -H^{-1}_c \nabla f(x)$ where $H_{c}$ is a model Hessian that is semi-positive definite will converge. The next algorithm will use the hessian if it is positive definite and the identity otherwise.

In [196]:
function armijo_alternating(f::Function, guess::Vector{Float64}; beta::Float64 = 0.9, 
    threshold::Float64 = 1e-8, tol::Float64 = 0.001, max_iter::Int64 = 50)

    gradient = x -> ForwardDiff.gradient(f, x)
    hessian = x -> ForwardDiff.hessian(f, x)
    
    #Intialize Guess_Array
    Guess_Array = Array{Any}(undef, max_iter+1)
    Guess_Array[1,1] = guess
    Newton_Steps::Int64 = 0
    Gradient_Steps::Int64 = 0

    for i in 2:max_iter+1
        H = hessian(guess)
        spd_bool = true
        eig_vals = eigvals(H)
        if minimum(eig_vals) <= 0.0
            spd_bool = false 
        end
        #Newton Step
        if spd_bool == true
            Newton_Steps += 1
            n::Int64 = 0
            val = f(Guess_Array[i-1])
            grad = gradient(Guess_Array[i-1])
            beta_n::Float64 = 1.0
            descent_direction = H\grad
            guess = Guess_Array[i-1] - descent_direction
            while true
                if f(Guess_Array[i-1])-f(guess) >= threshold
                    break
                end
                n += 1
                guess = Guess_Array[i-1] - beta ^ n .* descent_direction
                if n >= 50
                    println("""Iteration stalled \n
                           Current Derivative Error: $(norm(gradient(guess),2))\n
                           Current Guess: $(guess)\n
                           Newton Steps: $(Newton_Steps)\n
                           Gradient_Steps: $(Gradient_Steps)
                        """)
                    return Guess_Array[1:i-1]
                end
            end
        #Gradient Descent Step
        else
            Gradient_Steps += 1
            j::Int64 = 0
            val = f(Guess_Array[i-1])
            grad = gradient(Guess_Array[i-1])
            guess = Guess_Array[i-1] - grad
            while true
                if f(Guess_Array[i-1])-f(guess) >= threshold
                    break
                end
                j += 1
                guess = Guess_Array[i-1] - beta ^ j .* grad
                if j >= 50
                    println("""Iteration stalled \n
                           Current Derivative Error: $(norm(gradient(guess),2))\n
                           Current Guess: $(guess)\n
                           Newton Steps: $(Newton_Steps)\n
                           Gradient_Steps: $(Gradient_Steps)
                        """)
                    return Guess_Array[1:i-1]
                end
            end
        end
        Guess_Array[i] = guess
        if norm(gradient(guess),2) <= tol
            println("""Tolerance achieved\n 
            Current Derivative Error: $(norm(Guess_Array[i],2))\n
            Current Guess: $(Guess_Array[i])\n
            Newton Steps: $(Newton_Steps)\n
            Gradient_Steps: $(Gradient_Steps)
            """)
            return Guess_Array[1:i]
        end
    end
    println("""Tolerance not achieved\n 
    Current Derivative Error: $(norm(Guess_Array[end],2))\n
    Current Guess: $(Guess_Array[end])\n
    Newton Steps: $(Newton_Steps)\n
    Gradient_Steps: $(Gradient_Steps)
    """)
    return Guess_Array
end

armijo_alternating (generic function with 1 method)

In [198]:
nums = 2 .* rand(Float64, 3) .- 1
nums = nums/norm(nums,2)
println([norm(x,2) for x in armijo_alternating(ex,nums; max_iter = 1000)])
println("\nWithout the Newton Steps\n")
println([norm(x,2) for x in armijo(ex,nums; max_iter = 1000)])

Tolerance achieved
 
Current Derivative Error: 0.0002733036915201665

Current Guess: [-0.0002269651108198975, -7.739675926472095e-5, -0.00013111631448595867]

Newton Steps: 4

Gradient_Steps: 0

[1.0, 0.6666666666666669, 0.31372549019607865, 0.05159892418258675, 0.0002733036915201665]

Without the Newton Steps

[1.0, 0.8956125353985525, 0.8240661647794175, 0.7305118872863658, 0.5934410737480493, 0.514008920569273, 0.4620387465762699, 0.37192855223996374, 0.3199834403357785, 0.31808484791659647, 0.31542959614536936, 0.3117406332437549, 0.3066624145006328, 0.29975952700152014, 0.29053582927061883, 0.27848932342513466, 0.26321628218947885, 0.24456232302278036, 0.22278259664766673, 0.19863115887729688, 0.17329325090623535, 0.14814403809296445, 0.12443220108283004, 0.10304068188151136, 0.08441227681811815, 0.06861634097995985, 0.05547595116159741, 0.044688551447049474, 0.0359116444984319, 0.028812733346112503, 0.023093259780664163, 0.0184967818242527, 0.014808818386590521, 0.011852901012644

### Trust Region Methods
Trust region methods overcome the problems that line search methods encounter with non-spd approximate Hessians. In particular, a Newton trust region strategy allows the use of complete Hessian information even in regions where the Hessian has negative curvature. The specific trust region methods we will present effect a smooth transition from the steepest descent direction to the Newton direction in a way that gives the global convergence properties of steepest descent and the fast local convergence of Newton’s method. How do they work?

Suppose our guess is $x_i$. Then, we take a ball around $x_i$ with radius $r$. We then estimate the minimmum of the local quadratic model in the ball, as well as finding the minimum along the boundary of the ball. Then, we update the $x_{i+1}$ and radius $r$. The goal of these methods is to provide a smooth transition from gradient descent to Newtons Method. This gives us good performanc when far away from the region as well as speedy local convergence once the standard assumptions are met.
    